In [26]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:10pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:11pt;padding:4px;}
table.dataframe{font-size:10px;}
</style>
"""))

<b><font size="6" color="red">ch15.데이터베이스 연동</font></b>
# 1절. SQLite 데이터 베이스 연동
- SQLite 데이터베이스는 별도의 DBMS없이 SQL을 이용하여 DB 엑세스 가능한 디스크 기반 db
- SQLite는 프로토타입(시제품) 용도
- 프로젝트 단계 : 분석 - 설계 - 구현 - 테스트 - 고객에게 배포 - 유지보수
                  SQLite <-|-> Oracel.MySQL, MariaDB, mongoDB, ... 
- C 라이브러리
- DB browser for sqlite(https://sqlitebrowser.org/)
## 1.1 SQLite 패키지 load

In [28]:
import sqlite3
sqlite3.sqlite_version

'3.40.1'

In [29]:
import pandas as pd
pd.__version__

'1.5.3'

## 1.2 데이터 베이스 연결

In [70]:
# DB 연결
con = sqlite3.connect('data/ch15_exmple1.db')
con

In [31]:
# 커서객체(SQL문 실행 -> 결과 조회) 생성
cursor = con.cursor()
cursor

In [82]:
import os

db_path = 'data/ch15_exmple1.db'

# 파일이 존재하면 삭제
if os.path.exists(db_path):
    os.remove(db_path)
    print(f"{db_path} 파일 삭제 완료!")
else:
    print(f"{db_path} 파일이 존재하지 않습니다.")

PermissionError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'data/ch15_exmple1.db'

In [32]:
cursor.execute('''
    CREATE TABLE MEMBER1(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

### insert, update, delete 전송

In [33]:
cursor.execute('''
    INSERT INTO MEMBER1 VALUES('홍길동',20, 'H@H.com')
''')
print('수행 결과 행수 : ', cursor.rowcount)

수행 결과 행수 :  1


In [34]:
sql = "INSERT INTO MEMBER1 VALUES('홍길동',20, 'H@H.com')"
print(sql)

INSERT INTO MEMBER1 VALUES('홍길동',20, 'H@H.com')


In [49]:
sql = "INSERT INTO MEMBER1 VALUES('박길동',30, 'k@H.com')"
print(sql)
cursor.execute(sql)
print('수행 결과 행수 : ', cursor.rowcount)

INSERT INTO MEMBER1 VALUES('박길동',30, 'k@H.com')
수행 결과 행수 :  1


In [50]:
conn.commit() # (반대) : conn.rolldack() DML에서만 가능

### select 전송

In [54]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER1
''')

In [52]:
# SELECT 전송
#   select 문 실행결과를 받는 함수
#    cursor.fetchone() : 결과를 한행씩 받을 때(튜플)
#    cursor.fetchall() : 결과를 모두 받을 때 (튜플 list)
#    cursor.fetchmany(n) : 결과를 n행 받을 때(튜플 list)
#    cursor.description : header 내용을 포함한 내용들

# INSERT, UPDATE, DELETE 전송 : cursor.rowcount
print(cursor.fetchall())

[('홍길동', 21, 'H@H.com'), ('김길동', 31, 'k@H.com'), ('박길동', 31, 'k@H.com'), ('신길동', 31, 'k@H.com'), ('박길동', 31, 'k@H.com')]


In [45]:
print(cursor.fetchall())

[]


In [47]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER1 order by NEXTAGE
''')
members = cursor.fetchmany(3)
members

[('홍길동', 21, 'H@H.com'), ('김길동', 31, 'k@H.com'), ('박길동', 31, 'k@H.com')]

In [48]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER1 order by NEXTAGE
''')
members = cursor.fetchone()
members

('홍길동', 21, 'H@H.com')

In [55]:
cursor.fetchall()

[('홍길동', 21, 'H@H.com'),
 ('김길동', 31, 'k@H.com'),
 ('박길동', 31, 'k@H.com'),
 ('신길동', 31, 'k@H.com'),
 ('박길동', 31, 'k@H.com')]

In [57]:
# 데이터 한줄씩 가져오기
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER1 order by NEXTAGE
''')
members = []
while True:
    member1 = cursor.fetchone()
    if member1 is None:
        break
    print(member1)
    members.append({'name':member1[0], 'age':member1[1], 'email':member1[2]})
import pandas as pd
pd.DataFrame(members)

('홍길동', 21, 'H@H.com')
('김길동', 31, 'k@H.com')
('박길동', 31, 'k@H.com')
('신길동', 31, 'k@H.com')
('박길동', 31, 'k@H.com')


,name,age,email
0,홍길동,21,H@H.com
1,김길동,31,k@H.com
2,박길동,31,k@H.com
3,신길동,31,k@H.com
4,박길동,31,k@H.com


In [61]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER1 order by NEXTAGE
''')
members = cursor.fetchall()
df = pd.DataFrame(members, columns=[description[0] for description in cursor.description])
df

,NAME,NEXTAGE,EMAIL
0,홍길동,21,H@H.com
1,김길동,31,k@H.com
2,박길동,31,k@H.com
3,신길동,31,k@H.com
4,박길동,31,k@H.com


In [60]:
# select문을 수행한 필드 정보
cursor.description

(('NAME', None, None, None, None, None, None),
 ('NEXTAGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [62]:
[description[0] for description in cursor.description]

['NAME', 'NEXTAGE', 'EMAIL']

## 1.3 SQL구문에 파라미터 사용하기
- qmark(DB에 따라 불가한 경우 있음)
- named(추천)

In [63]:
conn = sqlite3.connect('data/ch15_exmple.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN('홍길동','김길동')")
cursor.fetchall()

[('홍길동', 20, 'H@H.com'), ('김길동', 30, 'k@H.com')]

In [66]:
# 파라미터 사용하기 : qmark 방법 이용
conn = sqlite3.connect('data/ch15_exmple.db')
cursor = conn.cursor()
sql = "SELECT * FROM MEMBER WHERE NAME IN(?,?)"
name1 = input('검색할 이름은?')
name2 = input('검색할 다른 이름은?')
cursor.execute(sql, (name1, name2))
cursor.fetchall()

검색할 이름은?홍길동
검색할 다른 이름은?성춘향


[('홍길동', 20, 'H@H.com')]

In [69]:
# 파라미터 사용하기 : named 방법 이용
sql = "SELECT * FROM MEMBER WHERE NAME IN(:name1,:name2)"
name1 = input('검색할 이름은?')
name2 = input('검색할 다른 이름은?')
cursor.execute(sql, {'name1':name1, 'name2':name2})
cursor.fetchall()

검색할 이름은?성춘향
검색할 다른 이름은?홍길동


[('홍길동', 20, 'H@H.com')]

In [72]:
# named 방법으로 데이터 입력하기
try:
    name = input('입력할 이름은 ?')
    age = int(input('입력할 나이은 ?'))
except ValueError:
    print('유효하지 않는 나이를 입력하셨습니다!!')
    age = 18
finally:
    email = input('입력할 메일은?')
cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)',
                {'name':name, 'age':age, 'email':email})
conn.commit()
if cursor.rowcount == 1:
    print('입력성공')
else:
    print('입력실패')

입력할 이름은 ?박길동
입력할 나이은 ?26
입력할 메일은?p@p.com
입력성공


In [81]:
cursor.close()
con.close()
conn.close()

# 2절. 오라클 데이터 베이스 연결
- pip install cx_oracle

In [76]:
import cx_Oracle
# conn 얻어오는 발법1
conn = cx_Oracle.connect('scott','tiger','localhost:1521/xe')
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [78]:
# conn 얻어오는 발법2
oracle_dsn = cx_Oracle.makedsn(host='localhost',port=1521, sid='xe')
conn = cx_Oracle.connect("scott","tiger",dsn = oracle_dsn)
conn

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [79]:
cursor = conn.cursor()
cursor.execute('SELECT EMPNO NO, ENAME, JOB, MGR, SAL, COMM FROM EMP')
emps = cursor.fetchall()
for emp in emps:
    print(emp)

(7369, 'SMITH', 'CLERK', 7902, 800.0, None)
(7499, 'ALLEN', 'SALESMAN', 7698, 1600.0, 300.0)
(7521, 'WARD', 'SALESMAN', 7698, 1250.0, 500.0)
(7566, 'JONES', 'MANAGER', 7839, 2975.0, None)
(7654, 'MARTIN', 'SALESMAN', 7698, 1250.0, 1400.0)
(7698, 'BLAKE', 'MANAGER', 7839, 2850.0, None)
(7782, 'CLARK', 'MANAGER', 7839, 2450.0, None)
(7788, 'SCOTT', 'ANALYST', 7566, 3000.0, None)
(7839, 'KING', 'PRESIDENT', None, 5000.0, None)
(7844, 'TURNER', 'SALESMAN', 7698, 1500.0, 0.0)
(7876, 'ADAMS', 'CLERK', 7788, 1100.0, None)
(7900, 'JAMES', 'CLERK', 7698, 950.0, None)
(7902, 'FORD', 'ANALYST', 7566, 3000.0, None)
(7934, 'MILLER', 'CLERK', 7782, 1300.0, None)


In [80]:
pd.DataFrame(emps, columns=[des[0]for des in cursor.description])

,NO,ENAME,JOB,MGR,SAL,COMM
0,7369,SMITH,CLERK,7902.0,800.0,NaN
1,7499,ALLEN,SALESMAN,7698.0,1600.0,300.0
2,7521,WARD,SALESMAN,7698.0,1250.0,500.0
3,7566,JONES,MANAGER,7839.0,2975.0,NaN
4,7654,MARTIN,SALESMAN,7698.0,1250.0,1400.0
5,7698,BLAKE,MANAGER,7839.0,2850.0,NaN
6,7782,CLARK,MANAGER,7839.0,2450.0,NaN
7,7788,SCOTT,ANALYST,7566.0,3000.0,NaN
8,7839,KING,PRESIDENT,NaN,5000.0,NaN
9,7844,TURNER,SALESMAN,7698.0,1500.0,0.0
